# Tarea 1


Notebooks adaptados y traducidos a partir de los tutoriales de código abierto 
utilizados en las competencias virtuales de los últimos 2 años
Yamamoto, T., Terada, K., Ochiai, A. et al. Development of Human Support Robot as the research platform of a domestic mobile manipulator. Robomech J 6, 4 (2019). https://doi.org/10.1186/s40648-019-0132-3

Por favor Consulte los originales en inglés y Japonés en https://github.com/hsr-project/hsrb_robocup_dspl_docker


Adaptado al españolp or Laboratorio BioRobótica UNAM
Oscar Fuentes


Para correr este notebook debe tenerse familiaridad con términos de ROS como tópicos, listeners, publishers, mensajes etc.


In [ ]:
#ROS NOETIC UBUNTU 18 PYTHON3 

#TEC

In [1]:
import rospy
from std_msgs.msg import String
import time
import numpy as np
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry
from tf.transformations import euler_from_quaternion


Importamos Librerías pertinentes para el control de la base del robot. 
Trabajamos a un alto nivel, es decir, confíamos que "under the hood" existe un control 
que permite traducir velocidades en señales de control

Atención especial al mensaje tipo Twist.
Un mensaje standar de ros dentro del paquete de mensajes geometry _msgs

In [ ]:
! rosmsg info geometry_msgs/Twist

el mensaje consta de 2 vectores de 3 dimensiones, uno llamado angular y y otro llamado linear

In [2]:
rospy.init_node('base_and_sensor')    ### Conectamos/creamos un nodo llamado base and sensor 
base_vel_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=10)## Declaramos un publisher que ha 
###de enviar mensajes tipo Twist al topico hsrb/command_velocity


In [3]:

def move_base_vel(vx, vy, vw):
    twist = Twist()
    twist.linear.x = vx
    twist.linear.y = vy
    twist.angular.z = vw 
    base_vel_pub.publish(twist)

Observe en el simulador y en RVIZ  al ingresar estos comandos

In [ ]:
move_base_vel(0.1, 0, 0)

In [ ]:
twist= Twist()

In [ ]:
twist.linear.x

In [ ]:
move_base_vel(0.0, 1.1, 0)

In [ ]:
move_base_vel(0 , 0, np.pi)

In [ ]:
move_base_vel(1, 1, np.pi*.25)

El robot HSR "Takeshi" tiene una base omnidireccional

Prepárese para interrumir el KERNEL con el pequeño  botón de STOP ubicado arriba en el notebook

Oops! The robot bumped into the wall. Press the ■ button above to stop our code.


In [ ]:
#puede mantenerse la velocidad deseada por ejemplo
while True:
    move_base_vel(1, 0, 0)

Ahora vamos a correr por un intervalo de tiempo definido la celda siguiente mantendrá una velocidad de 0.5m/s por 3 segundos:

In [ ]:
start_time = rospy.Time.now().to_sec()  
while rospy.Time.now().to_sec() - start_time < 3: 
    move_base_vel(-0.5, 0, 0)

EJERCICIO EN CLASE

In [15]:
def move_base(x,y,yaw,timeout=5):
    start_time = rospy.Time.now().to_sec()
    while rospy.Time.now().to_sec() - start_time < timeout:  
        move_base_vel(x, y, yaw)

TAREA:

Escriba un pequeño código que permita al robot volver al punto de partida de este tutorial, después de rodear el pequeño rover

In [17]:
pos_act = Twist()
def odom2pos(msg):
    pos_act.linear.x = msg.pose.pose.position.x
    pos_act.linear.y = msg.pose.pose.position.y
    pos_act.linear.z = msg.pose.pose.position.z
    #https://www.theconstructsim.com/ros-qa-053-how-to-move-a-robot-to-a-certain-point-using-twist/
    rot_q = msg.pose.pose.orientation
    (_,_,pos_act.angular.z) = euler_from_quaternion([rot_q.x, rot_q.y, rot_q.z, rot_q.w])
    


In [19]:
odom = rospy.Subscriber('/odom',Odometry, odom2pos)


linear: 
  x: -8.23083537118234e-05
  y: 0.00036452863249900687
  z: -0.0010013935255730917
angular: 
  x: 0.0
  y: 0.0
  z: 0.00574220992891022
linear: 
  x: -8.23083537118234e-05
  y: 0.00036452863249900687
  z: -0.0010013935255730917
angular: 
  x: 0.0
  y: 0.0
  z: 0.00574220992891022
linear: 
  x: -8.231182850120672e-05
  y: 0.00036454835240594353
  z: -0.0010013935256078937
angular: 
  x: 0.0
  y: 0.0
  z: 0.005742521254622471
linear: 
  x: -8.231182850120672e-05
  y: 0.00036454835240594353
  z: -0.0010013935256078937
angular: 
  x: 0.0
  y: 0.0
  z: 0.005742521254622471
linear: 
  x: -8.2315408602502e-05
  y: 0.0003645686699315553
  z: -0.0010013935256437574
angular: 
  x: 0.0
  y: 0.0
  z: 0.005742842015189911
linear: 
  x: -8.2315408602502e-05
  y: 0.0003645686699315553
  z: -0.0010013935256437574
angular: 
  x: 0.0
  y: 0.0
  z: 0.005742842015189911
linear: 
  x: -8.231888342151092e-05
  y: 0.0003645883899272659
  z: -0.0010013935256785837
angular: 
  x: 0.0
  y: 0.0
  z: 0.0

In [16]:
for i in range(4):
    move_base(0.2,0,0,timeout=5)
    #move_base(0,0,0,timeout=0.2)
    move_base(0,0,np.pi/8,timeout=4)    
    move_base(0,0,0,timeout=1)

move_base_vel(0, 0, 0)   

KeyboardInterrupt: 

Ahondaremos en el uso de tf en ros en sesiones futuras, por ahora utilicemos esta simple función para obtener de forma objetiva la posicionde inicio y final


In [ ]:
import tf2_ros
tfBuffer = tf2_ros.Buffer()
listener = tf2_ros.TransformListener(tfBuffer)
def get_coords ():
    trans = tfBuffer.lookup_transform('map', 'base_link', rospy.Time())
    return trans


In [ ]:
coords_start= get_coords()
move_base(0,0,.12*np.pi,4)

In [ ]:
move_base(0.05,0,0,0.5)
move_base(0.3,0,0,0.5)
move_base(0.4,0,0,4)

In [ ]:
move_base(0,0,.12*np.pi,4)

In [ ]:
move_base(0.05,0,0,0.5)
move_base(0.3,0,0,0.5)
move_base(0.4,0,0,0.5)
move_base(1,0,0,2.5)

In [ ]:
move_base(0,0,.12*np.pi,4)

In [ ]:
move_base(0.05,0,0,0.5)
move_base(0.3,0,0,0.5)
move_base(0.4,0,0,0.5)
move_base(1,0,0,6)

In [ ]:
move_base(0,0,.12*np.pi,4)

In [ ]:
move_base(0.05,0,0,0.5)
move_base(0.3,0,0,0.5)
move_base(0.4,0,0,0.5)
move_base(1,0,0,6)

In [ ]:
move_base(0,0,.12*np.pi,4)

In [ ]:
move_base(0.05,0,0,0.5)
move_base(0.3,0,0,0.5)
move_base(0.4,0,0,0.5)
move_base(1,0,0,6)

In [ ]:
coords_end= get_coords()
coords_start.transform.translation,coords_end.transform.translation

In [ ]:
print(coords_start.transform.translation,coords_end.transform.translation)

Sensores:
Si bien el HSR cuenta con muchos más sensores, (incluída la versión simulada) , por ahora nos concentramos en 2 sensores concretamente, que permiten extraer mucha información valiosa. 

Primero  EL LIDAR: Un conjunto de distancias distribuidas en un arco 270grados frente al robot
y luego camara rgbd xtion. Que nos da información en una imagen rgb común, y la nube de puntos relacionada. 

# Capturing sensor information
In the previous section, we controlled the movement of the robot using our prior knowledge about the environment. In this section, we will use sensors to acquire environmental information to move the robot.

We can use the following sensors installed in HSR:

- Laser scanner: Two-dimensional measurement of distance to obstacles

- RGB-D camera: A camera that can measure color information (RGB) + depth information (Depth)

- IMU: Measures acceleration, angular acceleration, and magnetic force

- Encoders: Measures each joint angle of the robot

## センサ情報の確認方法

## How to visualize sensor information

rvizを使ってセンサ情報を視覚的に確認してみましょう。

まずは、以下のコマンドを実行してrvizを起動します。

Let's check the sensor information visually using RViz.

First, launch RViz by executing the following command:

Primero, analicemos la información de los sensores usando RVIZ

In [ ]:
%%script bash --bg
rviz -d data/2_base_and_sensor.rviz > /dev/null 2>&1

En la ventana displays de RVIZ buscar y desplegar 

- RobotModel: Representación del estado del robot


- LaserScan: Sensor LIDAR 


- Image: imagen  rgb


- PointCloud2: Nube de Puntos

Please check the data you are interested in using the "Displays" panel on the left side of the screen. The meaning of each item is as follows:

- RobotModel: Robot CG model that reflects robot position estimation and joint angles.

- LaserScan: Two-dimensional distance to obstacles measured by the laser scanner.

- Image: Image from the RGB-D camera mounted on the head.

- PointCloud2: Point cloud information of the environment generated from the RGB-D camera.

Ejemplo de una clase que permite leer la información (tipo LaserScan) del topico '/hsrb/base_scan'

In [ ]:
from sensor_msgs.msg import LaserScan

class Laser():
    u"""Class that handles laser information"""

    def __init__(self):
        # Register the _laser_cb method as a callback to the laser scan topic events
        self._laser_sub = rospy.Subscriber ('/hsrb/base_scan',
                                           LaserScan, self._laser_cb)
        self._scan_data = None

    def _laser_cb (self, msg):
        # Laser scan callback function
        self._scan_data = msg

    def get_data(self):
        u"""Function to get the laser value"""
        return self._scan_data

In [ ]:
laser = Laser()  #instanciamos una clase 

以下を実行することで、データを取得することができます。取得されたセンサ値を、`scan_data`変数に格納しています。


Llamando en método `get_data()` almacenamos la información del sensor en la variable `scan_data`:


In [ ]:
scan_data = laser.get_data()

`scan_data`変数に格納されたセンサ値の中身を見てみましょう。

Analicemos la variable `scan_data` :

In [ ]:
scan_data


In [ ]:
scan_data

In [ ]:
##Ejemplo de como acceder a los datos del mensaje
scan_data.angle_max / np.pi * 180  # math.pi = π

レーザスキャナのデータ本体は、`ranges`という名前の配列に格納されており、配列の長さは721のようです。

The actual laser scanner data is stored in the array named `ranges`, and the length of the array seems to be 721:

La información del scanner se encuentra en el arreglo llamado `ranges`, su longitud es de  721:

In [ ]:
# データの配列の長さを取得 Get length of the array
len(scan_data.ranges)

スキャン範囲（-120度から120度）上の各観測点がデータ化されているので、配列の真ん中（=361番目）の要素が「ロボット正面から壁までの距離」を表します。

センサ値の単位はメートルです。

Since each observation point on the scan range (-120 degrees to 120 degrees) is digitized from left to right, the element in the middle of the array (= 361th) represents the "distance from the front of the robot to the wall".

The unit of sensor value is meters.

Ya que cada observación está dividida en un arco (-120 degrees to 120 degrees) el elemento en el centro del arreglo (= 361)representa la distancia frente al robot en metros



In [ ]:
# 361番目のデータにアクセス Access the 361th data
scan_data.ranges[360]

Tarea  utililce la información del sensores para mejorar el código que propuso en la tarea anterior. Como evitar chocar con la pared? Todos los obstáculos son visibles al robot?
A falta de mapa, como recordar la posición inicial

¡Buena Suerte!

In [ ]:
trans = tfBuffer.lookup_transform('map', 'base_link', rospy.Time())

In [ ]:
def set_waypoint ():
    trans = tfBuffer.lookup_transform('map', 'base_link', rospy.Time())
    return trans

trans_goal= set_waypoint()


In [ ]:
tans_start_point =set_waypoint()


In [ ]:
tans_start_point